## Learning Outcomes
- Exploratory data analysis & preparing the data for model building. 
- Machine Learning - Supervised Learning Classification
  - Logistic Regression
  - Naive bayes Classifier
  - KNN Classifier
  - Decision Tree Classifier
  - Random Forest Classifier
  - Ensemble methods
- Training and making predictions using different classification models.
- Model evaluation

## Objective: 
- The Classification goal is to predict “heart disease” in a person with regards to different factors given. 

## Context:
- Heart disease is one of the leading causes of death for people of most races in the US. At least 1 of 3 key risk factors for heart disease: high blood pressure, high cholesterol, and smoking. 
- Detecting and preventing the factors that have the greatest impact on heart disease is very important in healthcare. Machine learning methods may detect "patterns" from the data and can predict whether a patient is suffering from any heart disease or not..

## Dataset Information

#### Source: https://www.kaggle.com/datasets/kamilpytlak/personal-key-indicators-of-heart-disease?datasetId=1936563&sortBy=voteCount
Originally, the dataset come from the CDC and is a major part of the Behavioral Risk Factor Surveillance System (BRFSS), which conducts annual telephone surveys to gather data on the health status of U.S. residents. 

This dataset consists of eighteen columns
- HeartDisease: Respondents that have ever reported having coronary heart disease (CHD) or myocardial infarction (MI)
- BMI: Body Mass Index (BMI)
- Smoking: smoked at least 100 cigarettes in your entire life
- AlcoholDrinking: Heavy drinkers (adult men having more than 14 drinks per week and adult women having more than 7 drinks per week
- Stroke:Ever had a stroke?
- PhysicalHealth: physical health, which includes physical illness and injury
- MentalHealth: for how many days during the past 30 days was your mental health not good?
- DiffWalking: Do you have serious difficulty walking or climbing stairs?
- Sex: male or female?
- AgeCategory: Fourteen-level age category
- Race: Imputed race/ethnicity value
- Diabetic: diabetes?
- PhysicalActivity: Adults who reported doing physical activity or exercise during the past 30 days other than their regular job
- GenHealth: Would you say that in general your health is good, fine or excellent?
- SleepTime: On average, how many hours of sleep do you get in a 24-hour period?
- Asthma: you had asthma?
- KidneyDisease: Not including kidney stones, bladder infection or incontinence, were you ever told you had kidney disease?
- SkinCancer: Ever had skin cancer?

### 1. Importing Libraries

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier


### 2. Load the dataset and display a sample of five rows of the data frame.

In [ ]:
df = pd.read_csv('Dataset/heart_2020_cleaned.csv')
df.head()

### 3. Check the shape of the data (number of rows and columns). Check the general information about the dataframe using the .info() method.

In [ ]:
# shape
df.shape

In [ ]:
# information
df.info()

### 4. Check the statistical summary of the dataset and write your inferences.

In [ ]:
#statistical summary of the data
df.describe().T

In [ ]:
#statistical summary of the categorical data
df.describe(include='O')

### inferences
1. BMI - min 12, max - 95
2. avg mental health is for 7 days and min is 0 that means mantal health remains good for 30 days
3. Race has 6 unique categories
4. Most of the data is in Yes or No format

### 5. Check the percentage of missing values in each column of the data frame. Drop the missing values if there are any.

In [ ]:
df.isnull().sum()/len(df)*100

##### - No missing values so no need to drop any value

### 6. Check if there are any duplicate rows. If any drop them and check the shape of the dataframe after dropping duplicates.

In [ ]:
# getting no. of dupicated rows
count = len(df[df.duplicated()])
print(" There are "+ str(count) +" duplicate rows in data")

In [ ]:
#dropping duplicates
df.drop_duplicates(inplace=True)

In [ ]:
#shape after dropping duplicates
df.shape

In [ ]:
# getting no. of dupicated rows after deletion
count = len(df[df.duplicated()])
print(" There are "+ str(count) +" duplicate rows in data")

### 7. Check the distribution of the target variable (i.e. 'HeartDisease') and write your observations.

In [ ]:
#count of target variable
df['HeartDisease'].value_counts().plot(kind='pie',autopct='%1.0f%%')
plt.show()

###### 
- Unbalanced distribution
- The Data set is highly imbalanced. This could result in wrong predictions.
- We can do upscaling and downscaling to make it balance one , also check AOC , ROC score

### 8. Visualize the distribution of the target column 'Heart disease' with respect to various categorical features and write your observations.

In [ ]:
# Categorical features in the dataset
categorical_features = df.select_dtypes(include=[np.object])
categorical_features.columns

In [ ]:
i = 1
plt.figure(figsize = (30,25))
for feature in categorical_features:
    plt.subplot(6,3,i)
    sns.countplot(x = feature,hue = 'HeartDisease' , data = df)
    i +=1

### 9. Check the unique categories in the column 'Diabetic'. Replace 'Yes (during pregnancy)' as 'Yes' and 'No, borderline diabetes' as 'No'.

In [ ]:
#unique categories in column 'Diabetic'
df['Diabetic'].unique()

In [ ]:
df['Diabetic'] = df['Diabetic'].replace({'Yes (during pregnancy)':'Yes','No, borderline diabetes':'No'})

In [ ]:
#recheck
df['Diabetic'].value_counts()

### 10. For the target column 'HeartDiease', Replace 'No' as 0 and 'Yes' as 1. 

In [ ]:
## Replacing "No" as 0 and "Yes" as 1
df['HeartDisease'] = df['HeartDisease'].replace({'Yes':1,'No':0})

In [ ]:
## lets check
df['HeartDisease'].value_counts()

### 11. Label Encode the columns "AgeCategory", "Race", and "GenHealth". Encode the rest of the columns using dummy encoding approach.

In [ ]:
object_type = [i for i in df[['AgeCategory','Race','GenHealth']] if df.dtypes[i] == object]
object_type


le = LabelEncoder()

def encoder(df):
    for i in object_type:
        q = le.fit_transform(df[i].astype(str))  
        df[i] = q                               
        df[i] = df[i].astype(int)
encoder(df)

In [ ]:
## Dummy encoding the rest of the columns, since they have binary entries. 'Yes' or 'No'
df = pd.get_dummies(df,drop_first=True)

In [ ]:
#few samples after encoding.
df.head(2)

### 12. Store the target column (i.e.'HeartDisease') in the y variable and the rest of the columns in the X variable.

In [ ]:
#lets store the target column in the y variable and the rest of the colum in the x variable
x=df.drop('HeartDisease',axis=1)
y=df['HeartDisease']

### 13. Split the dataset into two parts (i.e. 70% train and 30% test) and print the shape of the train and test data

In [ ]:
## train_tes_split() is used to divide dataset into traning and test set
x_train,x_test,y_train,y_test= train_test_split(x,y,test_size=0.2)

print(x_train.shape,x_test.shape)
print(y_train.shape,y_test.shape)

### 14. Standardize the numerical columns using Standard Scalar approach for both train and test data.

In [ ]:
## Standardizing onlu numarical colums.(not standardizing the dummy encoded data)
ss = StandardScaler()

x_train.iloc[:,:7]=ss.fit_transform(x_train.iloc[:,:7])
x_test.iloc[:,:7]=ss.transform(x_test.iloc[:,:7])

In [ ]:
x_train

In [ ]:
x_test

### 15. Write a function.
- i) Which can take the model and data as inputs.
- ii) Fits the model with the train data.
- iii) Makes predictions on the test set.
- iv) Returns the Accuracy Score.

In [ ]:
def fit_n_print(model,x_train,x_test,y_train,y_test): # take the model, and data as inputs
    model.fit(x_train,y_train) # fits the model with the train data
    pred= model.predict(x_test) #makes prediction on the test set
    accuracy =accuracy_score(y_test,pred)
    return accuracy #return all the metrics|

### 16. Use the function and train a Logistic regression, KNN, Naive Bayes, Decision tree, Random Forest, Adaboost, GradientBoost, and Stacked Classifier models and make predictions on test data and evaluate the models, compare and write your conclusions and steps to be taken in future in order to improve the accuracy of the model.

In [ ]:
## intializing the models

lr = LogisticRegression()
nb = GaussianNB()
knn = KNeighborsClassifier()
dt = DecisionTreeClassifier()
rf = RandomForestClassifier()
adb = AdaBoostClassifier()
gb =  GradientBoostingClassifier()


estimators =[('rf',rf),('knn',knn),('gb',gb),('adb',adb)]
sc=StackingClassifier(estimators=estimators,final_estimator=rf)

In [ ]:
result = pd.DataFrame(columns=['Accuracy'])

for model, model_name in zip([lr, nb ,knn ,dt ,rf ,adb ,gb ,sc ],
                            ['Logistic Regression','Naive Bayes','KNN','Decision tree',
                            'Random Forest','Ada Boost','Gradient Boost','Stacking']):
    result.loc[model_name]=fit_n_print(model,x_train,x_test,y_train,y_test)

In [ ]:
result

### Conclusion

#### 
=> From EDA:-
- People who smoke have higher chance of heart disease then no smokers
- Male are more prone to suffer from heart disease as compare to females
- People with age limit in between 55-75 has higer chances of getting heart disease
- If people has difficulty in walking or climbing stairs there is a high probability that he or she is suffering from some heart disease
- There are higher chances of a person having heart disease if he does not have any kidney disease.

=> From Model:-
- We see that the best perfomrming models are gradient boost,adaboost,logistic regression and knn with around 91% accuracy.
- Please note that the target class is highly imbalanced so, accutracy does not hold good for imbalanced data.Accuracy becomes poor measure of evaluation for our classification model in the context.
- We can future go with evaluatin the modles based on the different perfomance metrics such as presicion ,recall and F1score and choose the model accordingly

----
## Happy Learning:)
----